In [ ]:
#TODO
#Hough transform
#Harris CD
#Blob detection
#pyramid

#Data prepare

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2

def imgshow(inimg):
    imgcv_rgb = cv2.cvtColor(inimg.copy(), cv2.COLOR_BGR2RGB)
    plt.imshow(imgcv_rgb)
    plt.show()

cat = cv2.imread('cat.jpg')
edge = cv2.imread('edge.jpg')
fox = cv2.imread('fox.jpg')

imgshow(edge)

In [ ]:
img = edge.copy()

height, width = img.shape[:2]
M = cv2.getRotationMatrix2D((width/2, height/2), 0, 0.8)
img = cv2.warpAffine(img, M,(width, height))

#Gray변환 후 엣지 검출
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150)

#허프 라인 검출, 직선으로 판단할 최소한의 동일 개수 = 150px
lines = cv2.HoughLines(edges,1,np.pi/180,150)
print(lines.shape) #(N, 1, 2)

for line in lines:
    rho, theta = line[0] #Distance, angle return
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho #x, y 절편 coord
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    #line 기준점 연산
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),1)
    
imgshow(img)

In [ ]:
img = edge.copy()

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150)
minLineLength = 100
maxLineGap = 0

lines = cv2.HoughLinesP(edges,1,np.pi/360,100,minLineLength,maxLineGap)
print(lines.shape) #(N, 1, 4 - x1, y1, x2, y2: 시작과 끝 좌표)

for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
        
imgshow(img)

In [ ]:
img = cat.copy()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.medianBlur(gray, 5)
circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1, 20,param1=50,param2=50,minRadius=100, maxRadius=0)

if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        # 원 둘레에 초록색 원 그리기
        cv2.circle(img,(i[0], i[1]), i[2], (0, 255, 0), 2)
        # 원 중심점에 빨강색 원 그리기
        cv2.circle(img, (i[0], i[1]), 2, (0,0,255), 5)
        
imgshow(img)

In [ ]:
img = edge.copy()

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
dst = cv2.cornerHarris(gray,2,3,0.04)

#Threshold for an optimal value
img[dst>0.1*dst.max()]=[0,0,255]
# 변화량 결과 최대값의 10% 이상의 좌표 빨갛게
# same as np.where(dst>0.1*dst.max())

imgshow(img)

In [ ]:
img = edge.copy()

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
dst = cv2.goodFeaturesToTrack(gray, 80, 0.01, 10)
print(dst.shape)
for corner in dst:
    corner = np.uint16(np.around(corner))
    cv2.circle(img, corner[0], 2, (0,0,255), 5)

imgshow(img)

In [ ]:
img = cat.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

detector = cv2.GFTTDetector_create() 
keypoint = detector.detect(gray, None)
imgfe = cv2.drawKeypoints(img, keypoint, None)

imgshow(imgfe)

In [ ]:
img = cat.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

detector = cv2.FastFeatureDetector_create()
keypoint = detector.detect(gray, None)
imgfe = cv2.drawKeypoints(img, keypoint, None)

imgshow(imgfe)

In [ ]:
img = cat.copy()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

detector = cv2.SimpleBlobDetector_create()
#SimpleBlobDetector_Params로 별도 지정 가능!
keypoint = detector.detect(gray, None)
imgfe = cv2.drawKeypoints(img, keypoint, None)

imgshow(imgfe)

In [ ]:
img = fox.copy()

down = cv2.pyrDown(img) # img x 1/4
up = cv2.pyrUp(img) # img x 4

imgarr = [img, down, up]

for i in imgarr:
    imgshow(i)
    print(i.shape)

In [ ]:
img = fox.copy()

down = cv2.pyrDown(img) # img x 1/4
up = cv2.pyrUp(down) # img x 4

imgshow(up)

In [ ]:
img = fox.copy()

down = cv2.pyrDown(img)
up = cv2.pyrUp(down)

# 원본에서 확대 subtract
laplacian = cv2.subtract(img, up)
# 확대 이미지에 라플라시안 더해서 복원
img = up + laplacian

imgshow(img)

In [ ]:
from skimage import data, io, filters

sfox = io.imread('fox.jpg')
io.imshow(sfox)
io.show()

In [ ]:
image = data.coins()
# ... or any other NumPy array!
edges = filters.sobel(image)
io.imshow(edges)
io.show()

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()

print(digits.images.shape)

_, axes = plt.subplots(nrows=1, ncols=10, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.imshow(image)
    ax.set_title("Im: %i" % label)

In [ ]:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1)) #flatten
print(data.shape)
X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.5, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from skimage.transform import warp, AffineTransform, ProjectiveTransform
from skimage.exposure import equalize_adapthist, equalize_hist, rescale_intensity, adjust_gamma, adjust_log, adjust_sigmoid
from skimage.filters import gaussian
from skimage.util import random_noise
import random

def randRange(a, b):
    return random.uniform(a, b)

def randomAffine(im):
    tform = AffineTransform(scale=(randRange(0.75, 1.3), randRange(0.75, 1.3)),
                            rotation=randRange(-0.25, 0.25),
                            shear=randRange(-0.2, 0.2),
                            translation=(randRange(-im.shape[0]//10, im.shape[0]//10), 
                                         randRange(-im.shape[1]//10, im.shape[1]//10)))
    return warp(im, tform.inverse, mode='reflect')

def randomPerspective(im):
    region = 1/4
    A = np.array([[0, 0], [0, im.shape[0]], [im.shape[1], im.shape[0]], [im.shape[1], 0]])
    B = np.array([[int(randRange(0, im.shape[1] * region)), int(randRange(0, im.shape[0] * region))], 
                  [int(randRange(0, im.shape[1] * region)), int(randRange(im.shape[0] * (1-region), im.shape[0]))], 
                  [int(randRange(im.shape[1] * (1-region), im.shape[1])), int(randRange(im.shape[0] * (1-region), im.shape[0]))], 
                  [int(randRange(im.shape[1] * (1-region), im.shape[1])), int(randRange(0, im.shape[0] * region))], 
                 ])

    pt = ProjectiveTransform()
    pt.estimate(A, B)
    return warp(im, pt, output_shape=im.shape[:2])

def randomIntensity(im):
    p_low, p_high = np.percentile(im, (1, 95))
    return rescale_intensity(im, in_range=(p_low, p_high))

def randomGamma(im):
    return adjust_gamma(im, gamma=randRange(0.5, 1.5))

def randomGaussian(im):
    return gaussian(im, sigma=random.randrange(0, 5))
    
def randomFilter(im):
    Filters = [equalize_adapthist, equalize_hist, adjust_log, adjust_sigmoid, randomGamma, randomGaussian, randomIntensity]
    filt = random.choice(Filters)
    return filt(im)

def randomNoise(im):
    var = randRange(0.001, 0.01)
    return random_noise(im, var=var)
    
def augment(im, Steps=[randomAffine, randomPerspective, randomFilter, randomNoise]):
    F = plt.figure(figsize=(15,9))
    G = plt.GridSpec(2, 3, figure=F)
    ax = plt.subplot(G[0])
    ax.imshow(im)
    ax.set_title('original')
    for i, step in enumerate(Steps):
        im = step(im)
        ax = plt.subplot(G[i+1])
        ax.imshow(im)
        ax.set_title(step.__name__)
    #return im

In [ ]:
augment(sfox.copy())

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

print("Train, Test len",len(trainset),len(testset))

train_real, valset = torch.utils.data.random_split(trainset, [40000,10000])

bat_size = 10
trainloader = torch.utils.data.DataLoader(train_real, batch_size=bat_size, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=bat_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=bat_size, shuffle=False, num_workers=2)
print("Train, Val, Test size with batch:",len(trainloader),len(valloader),len(testloader))

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(10)))

In [ ]:
#Code from DCGAN pytorch, utils.py

import torch
import torchvision.transforms as transforms
import torchvision.datasets as dset

def get_celeba(params):
    # Data proprecessing.
    transform = transforms.Compose([
        transforms.Resize(params['imsize']),
        transforms.CenterCrop(params['imsize']),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
            (0.5, 0.5, 0.5))])

    # Create the dataset.
    root = params['dataset']
    print(root)
    dataset = dset.ImageFolder(root=root, transform=transform)

    # Create the dataloader.
    dataloader = torch.utils.data.DataLoader(dataset,
        batch_size=params['bsize'],
        shuffle=True)

    return dataloader